## Importamos librerías.

In [2]:
import pandas as pd
from datetime import datetime
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
import numeritos as nitos
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
import os
import pickle 

pygame 2.1.2 (SDL 2.0.18, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Javier\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from nltk.corpus import stopwords
stopwords.words('spanish')[:5]

['de', 'la', 'que', 'el', 'en']

#### Recuperamos el modelado

In [16]:
os.getcwd()

'd:\\Bootcamp_22\\Javier\\5-Orientation\\Technical_test'

In [4]:
model=pickle.load( open('model/sentiment_model', "rb" ) )

C:\Users\Javier\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

C:\Users\Javier\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:329: UserWarning:

Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

C:\Users\Javier\AppData\Local\Packages\PythonSoftwareFoundation.Pytho

In [5]:
model

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.5, max_features=1000, min_df=5,
                                 ngram_range=(1, 2))),
                ('cls', LogisticRegression(C=0.5))])

En principio deberíamos de trabajar con los datos existentes en la BBDD, pero dado que hay que hacer transformaciones y éstas ya están realizadas en el anterior Notebook, partiremos de un fichero .csv del punto 4 y trabajaremos a partir de él. 

Extraemos la información de la Base de Datos a un DataFrame y cerramos la BBDD.

In [7]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [8]:
connection = sqlite3.connect("Twitter_bridge.db")
crsr = connection.cursor()
query = '''
SELECT * FROM TWEETS
;
'''
df_tweets = sql_query(query)
df_tweets.head()

,index,identity,text,created,user_id,quote,reply,retweet,favorite
0,2,1577720804619571201,RT @TheBridge_Tech: Ya están aquí nuestros pri...,2022-10-05 18:02:07+00:00,1099588676181471233,0,0,0,0
1,3,1577648996927012866,RT @TheBridge_Tech: ¡Feliz Día a nuestros Lead...,2022-10-05 13:16:46+00:00,763045308095950848,0,0,0,0
2,4,1577646040269864960,RT @TheBridge_Tech: ¡Feliz Día a nuestros Lead...,2022-10-05 13:05:01+00:00,1099588676181471233,0,0,0,0
3,5,1577335173523308544,RT @PauMugarra: Recordaros que la semana que v...,2022-10-04 16:29:45+00:00,7154412,0,0,0,0
4,6,1577334577701453827,Recordaros que la semana que viene tenemos la ...,2022-10-04 16:27:23+00:00,1003872445,0,0,2,12


In [9]:
connection.commit()
connection.close()

In [10]:
df_tweets

,index,identity,text,created,user_id,quote,reply,retweet,favorite
0,2,1577720804619571201,RT @TheBridge_Tech: Ya están aquí nuestros pri...,2022-10-05 18:02:07+00:00,1099588676181471233,0,0,0,0
1,3,1577648996927012866,RT @TheBridge_Tech: ¡Feliz Día a nuestros Lead...,2022-10-05 13:16:46+00:00,763045308095950848,0,0,0,0
2,4,1577646040269864960,RT @TheBridge_Tech: ¡Feliz Día a nuestros Lead...,2022-10-05 13:05:01+00:00,1099588676181471233,0,0,0,0
3,5,1577335173523308544,RT @PauMugarra: Recordaros que la semana que v...,2022-10-04 16:29:45+00:00,7154412,0,0,0,0
4,6,1577334577701453827,Recordaros que la semana que viene tenemos la ...,2022-10-04 16:27:23+00:00,1003872445,0,0,2,12
...,...,...,...,...,...,...,...,...,...
63,65,1567829433175756801,RT @PyDataMadrid: Ya está abierto el registro ...,2022-09-08 10:57:20+00:00,1162694149956603904,0,0,0,0
64,66,1567817628453949441,RT @PyDataMadrid: Ya está abierto el registro ...,2022-09-08 10:10:25+00:00,1367036537943244803,0,0,0,0
65,67,1567815140455710721,Ya está abierto el registro para nuestra reuni...,2022-09-08 10:00:32+00:00,1494355208473874438,2,1,5,8
66,68,1567802683360219136,.@TheBridge_Tech y @Fujitsu_ES se unen para in...,2022-09-08 09:11:02+00:00,581925228,0,0,0,1


#### 1.- INTERPRETACIÓN MODELO

In [11]:
from nltk.stem.snowball import SnowballStemmer

def spanish_stemmer(x):
    stemmer = SnowballStemmer('spanish')
    return " ".join([stemmer.stem(word) for word in x.split()])

df_tweets['text'] = df_tweets['text'].apply(spanish_stemmer)
df_tweets['text'].head()

0    rt @thebridge_tech: ya estan aqu nuestr primer...
1    rt @thebridge_tech: ¡feliz dia a nuestr lead i...
2    rt @thebridge_tech: ¡feliz dia a nuestr lead i...
3    rt @paumugarra: recordar que la seman que vien...
4    recordar que la seman que vien ten la primer q...
Name: text, dtype: object

In [12]:
predictions=model.predict(df_tweets['text'])
predictions

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0], dtype=int64)

In [13]:
df_tweets['Result'] = pd.Series(predictions)
df_tweets.head()

,index,identity,text,created,user_id,quote,reply,retweet,favorite,Result
0,2,1577720804619571201,rt @thebridge_tech: ya estan aqu nuestr primer...,2022-10-05 18:02:07+00:00,1099588676181471233,0,0,0,0,0
1,3,1577648996927012866,rt @thebridge_tech: ¡feliz dia a nuestr lead i...,2022-10-05 13:16:46+00:00,763045308095950848,0,0,0,0,0
2,4,1577646040269864960,rt @thebridge_tech: ¡feliz dia a nuestr lead i...,2022-10-05 13:05:01+00:00,1099588676181471233,0,0,0,0,0
3,5,1577335173523308544,rt @paumugarra: recordar que la seman que vien...,2022-10-04 16:29:45+00:00,7154412,0,0,0,0,1
4,6,1577334577701453827,recordar que la seman que vien ten la primer q...,2022-10-04 16:27:23+00:00,1003872445,0,0,2,12,1


In [34]:
df_tweets['Result'].value_counts(normalize=True)

0    0.632353
1    0.367647
Name: Result, dtype: float64

Los datos representan que 2/3 partes de los *Tweets* que se han publicado y donde se han mencionado a TheBridge_Tech, el modelo considera que no han tenido exito.

Considera que solo 1/3 parte de los *Tweets* han tenido un impacto importante en la comunidad de Twitter.

Habría que conocer también qué métricas de error hay, ya que hay algún resultado anómalo dentro de los Tweets que considera positivos.

#### 2.- VARIABLES

In [37]:
import eli5

In [61]:
variables=eli5.show_weights(estimator= model.named_steps['cls'],
                  vec=model.named_steps['vect'],
                  top=100)

C:\Users\Javier\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [66]:
variables

Weight?,Feature
+1.427,cuent
+1.402,deficit
+1.369,blanc
+1.350,muert
+1.343,conden
+1.342,denunci
+1.319,recort
+1.315,indign
+1.310,critic
+1.270,cos


Se puede observar que las *features* que aparecen en este modelo no tiene una relación aparente con el proyecto que llevamos a cabo.

Es por esto que los resultados obtenidos y como hemos comentado en el anterior punto, pueden no ser todo lo objetivos que deberían.

#### 3.- IMPROVEMENT

> Primeramente, habría que desarrollar un modelo que estuviera entrenado con variables relacionadas con @TheBridge_Tech, que es una empresa de formación dedicada a la tecnología y no con temas políticos.
>
> Seguidamente, habría que incluir en el estudio no sólo los textos, sino también añadir otras variables como son las métricas de públicas para evaluar si un Tweet es exitoso o no.
>
> Habría también que considerar la fecha de publicación de los *Tweets*, y comparar si en esa época del año la respuesta de los usuarios de Twitter, en general, es mayor o menor para evaluar periodos del año dónde es poco aconsejable tener incidencia en esta red. Y esto se puede incluir en un modelo.
 

#### 4.- OTHERS  OPPORTUNITIES

No tengo claro el sentido de este punto.

Si por __Qué otras oportunidades se te ocurren donde se podrían aplicar otros modelos de ML__ se entiende que otros posibles usos se pueden usar de modelado (sea con este modelo usado o con otro) en Twitter (por poner una comunidad), las oportunidades de negocio podrían ser:
* Determinar **periodos de menor/mayor actividad** para interactuar con los usuarios (en horas/días/meses).
* Tipo de mensaje más efectivo de cara a los usuarios, en función de lo que se quiera transmitir se podrían usar unas determinadas palabras u otras y así adecuar tus **campañas de comunicación**.
* Se puede saber qué tipo de respuesta se obtiene por parte de los usuarios, entendiendo por esto, si los **followers son activos** o no. El siguiente paso a esto debería de ser intentar que los followers sean lo más activos posibles, retuiteando mensajes, clickando en favoritos, etc.
* Paralelo a esto debería de incluirse si las actividades generadas en torno a @TheBridge_Tech son **propias** de nuestro cliente o si son externo al cliente.

Como hemos indicado en el punto anterior, todo esto se debería de incluir en una mejora del modelo de Machine Learning que estemos utilizando como variables predictoras para así poder hacer uso de un modelo de ML que sea aplicable y que mejore el actual.
